In [7]:
import socket
import struct
import time

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

# Crear socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)

# Permitir reutilización de direcciones
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Unirse al grupo de multidifusión en todas las interfaces
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

# Enlazar al puerto del grupo de multidifusión
sock.bind(('', MCAST_PORT))

# Configurar TTL
ttl = struct.pack('b', 1)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_MULTICAST_TTL, ttl)

# Enviar mensaje de saludo al grupo
sock.sendto(b"Hello, this is user 1", (MCAST_GRP, MCAST_PORT))

# Esperar y procesar mensajes entrantes hasta recibir "Adiós"
while True:
    try:
        data, addr = sock.recvfrom(1024)  # buffer size is 1024 bytes
        print(f"Received message: {data} from {addr}")

        if data.decode() == "Adiós":
            print("Received exit command, leaving multicast group")
            break
    except socket.error as e:
        print(f"Socket error: {e}")
        break

# Salir del grupo de multidifusión
sock.setsockopt(socket.IPPROTO_IP, socket.IP_DROP_MEMBERSHIP, mreq)
sock.close()
